In [20]:
import pandas as pd
import numpy as np
import os
import random
import shutil
import cv2

from PIL import Image

# Downloading datasets

In [21]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()
api.dataset_download_files('asdasdasasdas/garbage-classification', path='../data/kaggle_1', unzip=True)


Dataset URL: https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification


Modifica al file originale

In [22]:
api.dataset_download_files('mostafaabla/garbage-classification', path='../data/kaggle_2', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/mostafaabla/garbage-classification


Let's do some renaiming and refactoring:

In [23]:
os.rename('../data/kaggle_2/garbage_classification', '../data/kaggle_2/imgs')
shutil.move('../data/kaggle_1/Garbage classification/Garbage classification', '../data/kaggle_1/imgs')
shutil.rmtree('../data/kaggle_1/Garbage classification')

We will use zero-indexed files

In [24]:
os.remove('../data/kaggle_1/one-indexed-files.txt')
os.remove('../data/kaggle_1/one-indexed-files-notrash_train.txt')
os.remove('../data/kaggle_1/one-indexed-files-notrash_test.txt')
os.remove('../data/kaggle_1/one-indexed-files-notrash_val.txt')

In [25]:
annotations_og = pd.read_csv('../data/kaggle_1/zero-indexed-files.txt', sep=' ', header=None)
annotations_og.columns = ['file', 'class']

random.seed(42)

In [26]:
annotations_og

,file,class
0,glass1.jpg,0
1,glass2.jpg,0
2,glass3.jpg,0
3,glass4.jpg,0
4,glass5.jpg,0
...,...,...
2522,trash133.jpg,5
2523,trash134.jpg,5
2524,trash135.jpg,5
2525,trash136.jpg,5


In [27]:
def add_files(files, n, offset, class_name_src, class_name_dst):
    random.shuffle(files)
    
    files = files[:n]
    new_files = []
    
    for f, i in zip(files, np.arange(offset+1, offset+n+1)):
        shutil.copyfile(f'../data/kaggle_2/imgs/{class_name_src}/{f}', f'../data/kaggle_1/imgs/{class_name_dst}/{class_name_dst}{i}.jpg')
        new_files.append(f'{class_name_dst}{i}.jpg')
    
    return new_files

In [28]:
class_offset = {
    "cardboard": (403, 447, 2),
    "plastic": (482, 368, 3),
    "trash": (137,697, 5),
    "paper": (594, 256, 1),
    "metal": (410, 440, 4)
}

#offset, n, index

In [29]:
for c, i in class_offset.items():
    files = os.listdir(f'../data/kaggle_2/imgs/{c}')
    new_files = add_files(files, len(files), i[0], c, c)
    
    annotations_og = pd.concat([annotations_og, pd.DataFrame({'file': new_files, 'class': i[2]})])

In [30]:
i = 501
for c in ["green-glass", "brown-glass", "white-glass"]:
    files = os.listdir(f'../data/kaggle_2/imgs/{c}')
    new_files = add_files(files, 115, i, c, 'glass')
    annotations_og = pd.concat([annotations_og, pd.DataFrame({'file': new_files, 'class': 0})])
    i += 150

In [ ]:
annotations_og.sort_values(['class', 'file'], inplace=True)
annotations_og.reset_index(drop=True, inplace=True)
annotations_og.to_csv('../data/kaggle_1/annotations.csv', sep=' ', index=False, header=False)

# Train - test - val split

In [32]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(annotations_og, test_size=0.2)
train, val = train_test_split(train, test_size=0.25)

In [33]:
train.to_csv('../data/train.csv', sep=' ', index=False, header=False)
val.to_csv('../data/val.csv', sep=' ', index=False, header=False)
test.to_csv('../data//test.csv', sep=' ', index=False, header=False)

In [34]:
annotations_og.groupby('class').count()

,file
class,
0,846
1,1644
2,1294
3,1347
4,1179
5,834


Let's delete kaggle_2 folder:

In [35]:
annotations_og[annotations_og['file'] == 'glass1.jpg']

,file,class
0,glass1.jpg,0


In [36]:
shutil.rmtree('../data/kaggle_2')

In [37]:
shutil.move('../data/kaggle_1/imgs', '../data/')

'../data/imgs'

In [38]:
shutil.move('../data/kaggle_1/annotations.csv', '../data/')

'../data/annotations.csv'

In [39]:
shutil.rmtree('../data/kaggle_1')